In [10]:

weather = [ 'sunny', 'rainy', 'foggy', 'snowy' ]

transitions = {
    ('rainy', 'sunny'): 1,
    ('sunny', 'rainy'): 1,
    **{ ('foggy',w): 1/2 for w in weather if w != 'sunny' and w != 'foggy' },
    **{ ('snowy',w): 1/3 for w in weather if w != 'snowy' }
}

transitions

{('rainy', 'sunny'): 1,
 ('sunny', 'rainy'): 1,
 ('foggy', 'rainy'): 0.5,
 ('foggy', 'snowy'): 0.5,
 ('snowy', 'sunny'): 0.3333333333333333,
 ('snowy', 'rainy'): 0.3333333333333333,
 ('snowy', 'foggy'): 0.3333333333333333}

In [20]:
from graphviz import Digraph

dot = Digraph(format='png')
dot.attr(rankdir='LR')

with dot.subgraph() as c:
    c.attr(rank='same')
    c.node('rainy')
    c.node('sunny')
with dot.subgraph() as c:
    c.attr(rank='same')
    c.node('foggy')
    c.node('snowy')
 
from itertools import product

for (v1,v2) in product(weather,weather):
    if (v1,v2) in transitions.keys():
        dot.edge(v1,v2,f"{transitions[(v1,v2)]:.02f}")
dot.render('weather')

'weather.png'

In [49]:
import numpy as np
import numpy.linalg as npl

def transition_prob(v,w,dict):
    # get the probability for the transition v->w
    if (v,w) in dict.keys():
        return dict[(v,w)]
    else:
        return 0

p = np.array([[ transition_prob(v,w,transitions) for v in weather] for w in weather ])

p

array([[0.        , 1.        , 0.        , 0.33333333],
       [1.        , 0.        , 0.5       , 0.33333333],
       [0.        , 0.        , 0.        , 0.33333333],
       [0.        , 0.        , 0.5       , 0.        ]])

In [33]:
vals,vecs = npl.eig(p)
vals

array([ 1.        ,  0.40824829, -1.        , -0.40824829])

In [36]:
npl.matrix_power(p,100)

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.00000000e-01, 2.00000000e-01, 1.23719308e-39, 0.00000000e+00],
       [4.00000000e-01, 6.00000000e-01, 0.00000000e+00, 1.23719308e-39]])

In [64]:
new_transitions = {
    ('rainy', 'sunny'): 1,
    ('sunny', 'rainy'): 1/2,
    ('sunny', 'snowy'): 1/2,
    **{ ('foggy',w): 1/2 for w in weather if w != 'sunny' and w != 'foggy' },
    **{ ('snowy',w): 1/3 for w in weather if w != 'snowy' }
}

In [40]:

new_dot = Digraph(format='png')
new_dot.attr(rankdir='LR')

with new_dot.subgraph() as c:
    c.attr(rank='same')
    c.node('rainy')
    c.node('sunny')
with new_dot.subgraph() as c:
    c.attr(rank='same')
    c.node('foggy')
    c.node('snowy')
 
for (v1,v2) in product(weather,weather):
    if (v1,v2) in new_transitions.keys():
        new_dot.edge(v1,v2,f"{new_transitions[(v1,v2)]:.02f}")
new_dot.render('new_weather')

'new_weather.png'

In [65]:

q = np.array([[ transition_prob(v,w,new_transitions) for v in weather] for w in weather ])

q

array([[0.        , 1.        , 0.        , 0.33333333],
       [0.5       , 0.        , 0.5       , 0.33333333],
       [0.        , 0.        , 0.        , 0.33333333],
       [0.5       , 0.        , 0.5       , 0.        ]])

In [72]:
npl.matrix_power(q,100)

array([[0.38461538, 0.38461538, 0.38461538, 0.38461538],
       [0.30769231, 0.30769231, 0.30769231, 0.30769231],
       [0.07692308, 0.07692308, 0.07692308, 0.07692308],
       [0.23076923, 0.23076923, 0.23076923, 0.23076923]])

In [67]:
vals,vecs = npl.eig(q)
vals

array([ 1.        , -0.78867513, -0.21132487,  0.        ])

In [70]:
# get the eigenvector computed by numpy for eigenvalue 1
ev = vecs[:,0]

# normalize to make a probability vector
c = np.array([1,1,1,1]) @ ev
pev = (1/c)*ev

pev

array([0.38461538, 0.30769231, 0.07692308, 0.23076923])